<a href="https://colab.research.google.com/github/first-penguin-hub/rabbit_challenge/blob/main/dcgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## Google Colab 用の処理

下記を実行します
- ドライブのマウント
- ノートブックファイルと同じフォルダへの移動

Googleドライブのマイドライブ を基準に DNN_code/DNN_code_colab_day4 フォルダを置くことを仮定しています。必要に応じて，パスを変更してください．

In [ ]:
# Google Colab での実行かを調べる
import sys
import os
ENV_COLAB = True  if 'google.colab' in sys.modules else False

# google drive のマウント
if ENV_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/StudyAI/DNN_code_colab_day4/notebook/4_10_code_dcgan')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# DCGAN による画像生成

In [ ]:
import glob
from pathlib import Path

import numpy as np
from PIL import Image

from model_dcgan import DCGAN


In [ ]:
def convert_img_to_npy(
    img_dir='"./img_data', cond="*.png", save_path="./train_data/train_data"
    ):
    """画像ファイル群を学習データ用にnumpy形式に変換する関数
    学習データが既にあれば実行する必要無し

    Args:
        img_dir (str): 画像ファイルがあるフォルダ. Defaults to "./img_data".
        cond (str): 変換する画像の条件. Defaults to "*.png".
        save_path (str): 変換後の保存先. Defaults to "./train_data/train_data".
    """
    dataset = []
    img_path = str(Path(img_dir) / cond)
    print("Loading Image Data from " + img_dir)
    for idx, img_path in enumerate(glob.glob(img_path)):
        # (height,width,channels)
        img_data = Image.open(img_path)
        print(str(idx) + " data loaded")
        # (1,height,width,channels)
        img_data = np.expand_dims(img_data, axis=0)
        if idx == 0:
            dataset = img_data
        else:
            dataset = np.concatenate([dataset, img_data], axis=0)

    dataset = (dataset - 127.5) / 127.5
    np.save(save_path, dataset)
    print("Train Data Saved")

convert_img_to_npy()

In [ ]:
args_dict = {
    'exp_name': "exp_2", # 実験名(この名前のフォルダが./expに生成される)
    "noize_dim": 100, # Gの入力乱数ベクトルの次元
    "batch_size": 128, # バッチサイズ
    "num_epoch": 100, # 学習エポック数
    "make_logs": True, # D,GのLossをLogとして記録するかどうか
    "restore_ckpt": False, # 学習済みモデルのckptを読み込むかどうか
    "sample_freq": 10, # 学習中の画像生成頻度(エポック数)
    "ckpt_freq": 10, # 学習中のckpt保存頻度(エポック数)
    "adam": {"lr":2e-4, # 学習率
            "beta_1":0.5,
            "beta_2":0.9,
            "epsilon":1e-8}
    }

In [ ]:
train_data_path = "./train_data/train_data.npy"
# 学習データサイズ(data_num, H:64, W:64, RGB:3), 値の範囲は[-1,1]
print("Loading Train Data")
train_data = np.load(train_data_path)
print("Train Data Loaded")

In [ ]:
model = DCGAN(args_dict)

In [ ]:
model.train(train_data)

In [ ]:
# 学習済みモデルを用いて画像生成
model.generate_img()